In [2]:
import os
import numpy as np

from mnucosmomap import util as UT
from mnucosmomap import catalogs as mNuCats

In [17]:
%load_ext line_profiler

In [18]:
fullbox = mNuCats.mNuICs(1, sim='paco')
x, y, z = fullbox['Position'].T
vx, vy, vz = fullbox['Velocity'].T

nside = 8
L_subbox = 1000./float(nside) # L_subbox
L_res = 1000./512.
L_halfres = 0.5 * L_res
N_partside = 512/nside
N_subbox = (N_partside)**3

In [10]:
def method1(isubbox): 
    i_x = ((isubbox % nside**2) % nside) 
    i_y = ((isubbox % nside**2) // nside) 
    i_z = (isubbox // nside**2) 

    xmin = L_subbox * float(i_x) + L_halfres
    xmax = (L_subbox * float(i_x+1) + L_halfres) % 1000.
    ymin = L_subbox * float(i_y) + L_halfres
    ymax = (L_subbox * float(i_y+1) + L_halfres) % 1000.
    zmin = L_subbox * float(i_z) + L_halfres
    zmax = (L_subbox * float(i_z+1) + L_halfres) % 1000.
    if xmin <= xmax: xlim = ((x >= xmin) & (x < xmax))
    else: xlim = ((x >= xmin) | (x < xmax))
    if ymin <= ymax: ylim = ((y >= ymin) & (y < ymax))
    else: ylim = ((y >= ymin) | (y < ymax))
    if zmin <= zmax: zlim = ((z >= zmin) & (z < zmax))
    else: zlim = ((z >= zmin) | (z < zmax))
    in_subbox = (xlim & ylim & zlim)
    assert np.sum(in_subbox) == N_subbox

    ID_sub = fullbox['ID'][in_subbox]
    x_subbox = x[in_subbox]
    y_subbox = y[in_subbox]
    z_subbox = z[in_subbox]
    x_sub = (x_subbox - i_x * L_subbox) % 1000.
    y_sub = (y_subbox - i_y * L_subbox) % 1000.
    z_sub = (z_subbox - i_z * L_subbox) % 1000.

    vx_subbox = vx[in_subbox]
    vy_subbox = vy[in_subbox]
    vz_subbox = vz[in_subbox]
    subbox_ID = np.zeros((N_partside, N_partside, N_partside))
    subbox_pos = np.zeros((3, N_partside, N_partside, N_partside))
    subbox_vel = np.zeros((3, N_partside, N_partside, N_partside))
    for j_z in range(N_partside): 
        #print('j_z = %i , %f < z < %f' % (j_z, L_res* float(j_z) + L_halfres, L_res * float(j_z + 1) + L_halfres))
        zlim_sub = ((z_sub > L_res* float(j_z) + L_halfres) & 
                (z_sub < L_res * float(j_z + 1) + L_halfres))
        for j_y in range(N_partside): 
            #print('j_y = %i , %f < y < %f' % (j_y, L_res* float(j_y) + L_halfres, L_res * float(j_y + 1) + L_halfres))
            ylim_sub = ((y_sub > L_res * float(j_y) + L_halfres) & 
                    (y_sub < L_res * float(j_y + 1) + L_halfres))
            #for j_x in range(N_partside): 
            j_x_sorted = np.argsort(x_sub[ylim_sub & zlim_sub])
            subbox_ID[:,j_y,j_z] = ID_sub[ylim_sub & zlim_sub][j_x_sorted]
            subbox_pos[0,:,j_y,j_z] = x_subbox[ylim_sub & zlim_sub][j_x_sorted]
            subbox_pos[1,:,j_y,j_z] = y_subbox[ylim_sub & zlim_sub][j_x_sorted]
            subbox_pos[2,:,j_y,j_z] = z_subbox[ylim_sub & zlim_sub][j_x_sorted]
            subbox_vel[0,:,j_y,j_z] = vx_subbox[ylim_sub & zlim_sub][j_x_sorted]
            subbox_vel[1,:,j_y,j_z] = vy_subbox[ylim_sub & zlim_sub][j_x_sorted]
            subbox_vel[2,:,j_y,j_z] = vz_subbox[ylim_sub & zlim_sub][j_x_sorted]
    return None

In [11]:
def method2(isubbox): 
    i_x = ((isubbox % nside**2) % nside) 
    i_y = ((isubbox % nside**2) // nside) 
    i_z = (isubbox // nside**2) 

    xmin = L_subbox * float(i_x) + L_halfres
    xmax = (L_subbox * float(i_x+1) + L_halfres) % 1000.
    ymin = L_subbox * float(i_y) + L_halfres
    ymax = (L_subbox * float(i_y+1) + L_halfres) % 1000.
    zmin = L_subbox * float(i_z) + L_halfres
    zmax = (L_subbox * float(i_z+1) + L_halfres) % 1000.
    if xmin <= xmax: xlim = ((x >= xmin) & (x < xmax))
    else: xlim = ((x >= xmin) | (x < xmax))
    if ymin <= ymax: ylim = ((y >= ymin) & (y < ymax))
    else: ylim = ((y >= ymin) | (y < ymax))
    if zmin <= zmax: zlim = ((z >= zmin) & (z < zmax))
    else: zlim = ((z >= zmin) | (z < zmax))
    in_subbox = (xlim & ylim & zlim)
    assert np.sum(in_subbox) == N_subbox

    ID_sub = fullbox['ID'][in_subbox]
    x_subbox = x[in_subbox]
    y_subbox = y[in_subbox]
    z_subbox = z[in_subbox]
    x_sub = (x_subbox - i_x * L_subbox) % 1000.
    y_sub = (y_subbox - i_y * L_subbox) % 1000.
    z_sub = (z_subbox - i_z * L_subbox) % 1000.

    vx_subbox = vx[in_subbox]
    vy_subbox = vy[in_subbox]
    vz_subbox = vz[in_subbox]
    
    subbox_ID = np.zeros((N_partside, N_partside, N_partside))
    subbox_pos = np.zeros((3, N_partside, N_partside, N_partside))
    subbox_vel = np.zeros((3, N_partside, N_partside, N_partside))

    j_x = ((x_sub - L_halfres) // L_res).astype(int) 
    j_y = ((y_sub - L_halfres) // L_res).astype(int) 
    j_z = ((z_sub - L_halfres) // L_res).astype(int) 
    subbox_ID[j_x,j_y,j_z] = ID_sub
    subbox_pos[0,j_x,j_y,j_z] = x_subbox
    subbox_pos[1,j_x,j_y,j_z] = y_subbox
    subbox_pos[2,j_x,j_y,j_z] = z_subbox
    subbox_vel[0,j_x,j_y,j_z] = vx_subbox
    subbox_vel[1,j_x,j_y,j_z] = vy_subbox
    subbox_vel[2,j_x,j_y,j_z] = vz_subbox
    return None

In [26]:
%timeit -n 3 method1(0)

3 loops, best of 3: 6.43 s per loop


In [27]:
%timeit -n 3 method2(0)

3 loops, best of 3: 2.21 s per loop
